In [25]:
import pandas as pd    
import geopandas as gpd
from geopandas.tools import geocode
import numpy as np
import scipy.stats as stats
import scipy
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

In [2]:
import urllib.request as request

**Imports all traffic data from 2007 - 2018**

Link: https://data.seattle.gov/browse?q=traffic%20flow%20count&sortBy=relevance to download yearly data

In [26]:
url_list = ['https://opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.geojson',
            'https://opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.geojson',
            'https://opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.geojson',
            'https://opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.geojson',
            'https://opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.geojson',
            'https://opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.geojson',
            'https://opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.geojson',
            'https://opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.geojson',
            'https://opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.geojson',
            'https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson',
            'https://opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.geojson',
            'https://opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.geojson']
    

In [64]:
def get_gdf(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    gdf_year = gpd.read_file(url_list[num])
    if year == 11:
        gdf_year = gdf_year.rename(columns={"YEAR_" : 'YEAR'})
    if year == 12:
        gdf_year = gdf_year.rename(columns={'STDY_YEAR' : 'YEAR'})
    if year == 15 or year == 16:
        gdf_year = gdf_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 15:
            year_list = ['2015']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 16:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    elif year == 17 or year == 18:
        gdf_year = gdf_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 17:
            year_list = ['2017']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 18:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    #df_year_AAWDT = df_year['AAWDT'].values
    #df_year_geobase = df_year['GEOBASID'].values
    #df_year_dist = df_year['SHAPE_Length'].values
    gdf_year = gdf_year[[ 'YEAR', 'AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
    return gdf_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [17]:
montlake_br_traffic_07 = df_07.loc[df_07['STNAME']=='MONTLAKE BR']

In [19]:
montlake_br_traffic_07

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,INPUT_STUD,HPMS,FLOW_COUNT,SHAPE_Length
197,198,86850010,N,11902,MONTLAKE BR,83,"MONTLAKE BR, S/O POINT A",2007,11902,57203,0,,Y,1362.081215


In [49]:
df_048 = get_df(8)

In [5]:
def get_montlake_br_data(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    df_year = get_df(year)
    montlake_br_traffic_year = df_year.loc[df_year['STNAME'] == 'MONTLAKE BLVD NE']
    return montlake_br_traffic_year

In [35]:
montlake_br_data_08 = get_montlake_br_data(8)

In [36]:
montlake_br_data_08['SHAPE_Length']

149    3751.859475
Name: SHAPE_Length, dtype: float64

In [6]:
montlake_br_data_by_year = pd.DataFrame()
year_list = range(7,19)
for year in year_list:
    num = year-7
    print(year)
    new = get_montlake_br_data(year)
    montlake_br_data_by_year = montlake_br_data_by_year.append(new)

7
8
9
10
11
12
13
14
15
16
17
18


In [7]:
montlake_br_data_by_year['SHAPE_Length']

148     3751.859475
149     3751.859475
149     3751.859475
149     3751.859475
126     2966.829559
108     2966.829559
155     3751.859475
157     3751.859475
561     2966.829559
572      575.275702
575      495.856467
695      785.029916
591     2966.829559
602      575.275702
605      495.856467
699      785.029916
710      495.856467
939      785.029916
1318     575.275702
1732    2966.829559
714      495.856467
864      785.029916
1236     575.275702
1711    2966.829559
Name: SHAPE_Length, dtype: float64

In [9]:
data = gpd.GeoDataFrame(montlake_br_data_by_year)

In [12]:
montlake_br_data_by_year_geo = geocode(data['STNAME'], provider = 'nominatim', user_agent='greening_sea', timeout=8)

In [13]:
montlake_br_data_by_year_geo.head()

,geometry,address
148,POINT (-122.30462 47.64781),"Montlake Boulevard Northeast, Montlake, Seattl..."
149,POINT (-122.30462 47.64781),"Montlake Boulevard Northeast, Montlake, Seattl..."
126,POINT (-122.30462 47.64781),"Montlake Boulevard Northeast, Montlake, Seattl..."
108,POINT (-122.30462 47.64781),"Montlake Boulevard Northeast, Montlake, Seattl..."
155,POINT (-122.30462 47.64781),"Montlake Boulevard Northeast, Montlake, Seattl..."


In [15]:
print(montlake_br_data_by_year_geo.crs)

EPSG:4326


In [16]:
print(data.crs)

None
